In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
torch.manual_seed(8) # for reproduce

import time
import numpy as np
import gc
import sys
sys.setrecursionlimit(50000)
import pickle
torch.backends.cudnn.benchmark = True
torch.set_default_tensor_type('torch.cuda.FloatTensor')
# from tensorboardX import SummaryWriter
torch.nn.Module.dump_patches = True
import copy
import pandas as pd
#then import my own modules
from AttentiveFP import Fingerprint, Fingerprint_viz, save_smiles_dicts, get_smiles_dicts, get_smiles_array, moltosvg_highlight

In [2]:
from rdkit import Chem
from rdkit.Chem import QED,MACCSkeys,rdMolDescriptors
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
%matplotlib inline
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from IPython.display import SVG, display
import seaborn as sns; sns.set(color_codes=True)

In [3]:
def load_model(method,fp):
    f= open('models/'+str(method)+'_'+str(fp)+'.model','rb')
    model = pickle.load(f)
    f.close()
    return model

def MACCS_FP(dataset, model = False):
    MACCS_FP = pd.DataFrame()
    if model == True:
        for smi,endpoint in zip(dataset.smiles,dataset.endpoint):
            mol = Chem.MolFromSmiles(smi)
            MA = MACCSkeys.GenMACCSKeys(mol).ToBitString()
            MACCS = [int(i) for i in MA]
            MACCS.append(endpoint)
            MACCS_FP = pd.concat([MACCS_FP,pd.DataFrame(MACCS).T])
        MACCS_FP.drop(columns=[0],axis=1,inplace = True)
    else:
        for smi in dataset.smiles:
            mol = Chem.MolFromSmiles(smi)
            MA = MACCSkeys.GenMACCSKeys(mol).ToBitString()
            MACCS = [int(i) for i in MA]
            MACCS_FP = pd.concat([MACCS_FP,pd.DataFrame(MACCS).T])
        MACCS_FP.drop(columns=[0],axis=1,inplace = True)
    return MACCS_FP

def RDK_FP(dataset,model = False):
    RDK_FP = pd.DataFrame()
    if model == True:
        for smi,endpoint in zip(dataset.smiles,dataset.endpoint):
            mol = Chem.MolFromSmiles(smi)
            RD = Chem.RDKFingerprint(mol).ToBitString()
            RDK = [int(i) for i in RD]
            RDK.append(endpoint)
            RDK_FP = pd.concat([RDK_FP,pd.DataFrame(RDK).T])
    else:
        for smi in dataset.smiles:
            mol = Chem.MolFromSmiles(smi)
            RD = Chem.RDKFingerprint(mol).ToBitString()
            RDK = [int(i) for i in RD]
            RDK_FP = pd.concat([RDK_FP,pd.DataFrame(RDK).T])
    return RDK_FP

def ECFP_FP(dataset, model = False):
    ECFP_FP = pd.DataFrame()
    if model == True:
        for smi,endpoint in zip(dataset.smiles,dataset.endpoint):
            mol = Chem.MolFromSmiles(smi)
            EC = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2,nBits=1024).ToBitString()
            ECFP = [int(i) for i in EC]
            ECFP.append(endpoint)
            ECFP_FP = pd.concat([ECFP_FP,pd.DataFrame(ECFP).T])
    else:
        for smi in dataset.smiles:
            mol = Chem.MolFromSmiles(smi)
            EC = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2,nBits=1024).ToBitString()
            ECFP = [int(i) for i in EC]
            ECFP_FP = pd.concat([ECFP_FP,pd.DataFrame(ECFP).T])
    return ECFP_FP

In [4]:
random_seed = 8
task_name = 'ames'
tasks = ['endpoint']
batch_size = 128
weight_decay = 4
learning_rate = 4
radius = 4
T = 2
p_dropout = 0.1
fingerprint_dim = 150
per_task_output_units_num = 2
output_units_num = len(tasks) * per_task_output_units_num
init_df = pd.read_csv('models/initialization_df.csv')
weights = []
for i,task in enumerate(tasks):    
    negative_df = init_df[init_df[task] == 0][["smiles",task]]
    positive_df = init_df[init_df[task] == 1][["smiles",task]]
    weights.append([(positive_df.shape[0]+negative_df.shape[0])/negative_df.shape[0],\
                    (positive_df.shape[0]+negative_df.shape[0])/positive_df.shape[0]])


loss_function = [nn.CrossEntropyLoss(torch.Tensor(weight),reduction='mean') for weight in weights]

def predict_prob(best_model,test,y_value = True):
    'Return the performance of the test validation'
    if y_value == False:
        test_x = pd.DataFrame(test.iloc[:,:]).values
        y_preds = best_model.predict(test_x)
        y_scores =  best_model.predict_proba(test_x)[:,1]
    else:
        test_x = pd.DataFrame(test.iloc[:,:-1]).values
        test_y = pd.DataFrame(test.iloc[:,-1]).values
        y_preds = best_model.predict(test_x)
        y_scores =  best_model.predict_proba(test_x)[:,1]
    return y_scores,y_preds

def gnn_predict_prob(model, dataset,test_df_feature_dicts):
    model.eval()
    y_val_list = {}
    y_pred_list = {}
    losses_list = []
    valList = np.arange(0,dataset.shape[0])
    batch_list = []
    for i in range(0, dataset.shape[0], batch_size):
        batch = valList[i:i+batch_size]
        batch_list.append(batch)
    for counter, eval_batch in enumerate(batch_list):
        batch_df = dataset.loc[eval_batch,:]
        smiles_list = batch_df.cano_smiles.values
        
        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(smiles_list,test_df_feature_dicts)
        atoms_prediction, mol_prediction = model(torch.Tensor(x_atom),torch.Tensor(x_bonds),torch.cuda.LongTensor(x_atom_index),torch.cuda.LongTensor(x_bond_index),torch.Tensor(x_mask))
        atom_pred = atoms_prediction.data[:,:,1].unsqueeze(2).cpu().numpy()
        for i,task in enumerate(tasks):
            y_pred = mol_prediction[:, i * per_task_output_units_num:(i + 1) *
                                    per_task_output_units_num]
            y_val = batch_df[task].values

            validInds = np.where((y_val==0) | (y_val==1))[0]
#             validInds = np.where((y_val=='0') | (y_val=='1'))[0]
#             print(validInds)
            if len(validInds) == 0:
                continue
            y_val_adjust = np.array([y_val[v] for v in validInds]).astype(float)
            validInds = torch.cuda.LongTensor(validInds).squeeze()
            y_pred_adjust = torch.index_select(y_pred, 0, validInds)
#             print(validInds)
            loss = loss_function[i](
                y_pred_adjust,
                torch.cuda.LongTensor(y_val_adjust))
#             print(y_pred_adjust)
            y_pred_adjust = F.softmax(y_pred_adjust,dim=-1).data.cpu().numpy()[:,1]
            losses_list.append(loss.cpu().detach().numpy())
            try:
                y_val_list[i].extend(y_val_adjust)
                y_pred_list[i].extend(y_pred_adjust)
            except:
                y_val_list[i] = []
                y_pred_list[i] = []
                y_val_list[i].extend(y_val_adjust)
                y_pred_list[i].extend(y_pred_adjust)
    
    return  y_pred_list

In [5]:
def consensus_model_predict(gnn_model,rf_rdk_model,svm_ecfp_model,lgb_rdk_model,xgb_maccs_model,gbt_maccs_model,dataset):
    MACCSFP = MACCS_FP(dataset)
    RDKFP = RDK_FP(dataset)
    ECFPFP = ECFP_FP(dataset)
    
    # base model
    rf_prob = predict_prob(rf_rdk_model, RDKFP,y_value = False)[0]
    svm_prob = predict_prob(svm_ecfp_model,ECFPFP,y_value = False)[0]
    lgb_prob = predict_prob(lgb_rdk_model,RDKFP,y_value = False)[0]
    xgb_prob = predict_prob(xgb_maccs_model,MACCSFP,y_value = False)[0]
    gbt_prob = predict_prob(gbt_maccs_model,MACCSFP,y_value = False)[0]
    
    dataset_filname = 'predict_data'
    dataset_df_feature_dicts = save_smiles_dicts([s for s in dataset.cano_smiles],dataset_filname)
    gnn_prob = gnn_predict_prob(gnn_model,dataset,dataset_df_feature_dicts)[0]
    
    # consensus model
    predict_proba = np.zeros((len(dataset),6))
    predict_proba = pd.DataFrame(predict_proba)
    consensus_model = load_model('stacking','6')
#    consensus_model = pickle.load('stacking_6_model.model')
    predict_proba.columns = ['rf','svm','lgb','xgb','gbt','gnn']
    predict_proba['rf'] = rf_prob
    predict_proba['svm'] = svm_prob
    predict_proba['lgb'] = lgb_prob
    predict_proba['xgb'] = xgb_prob
    predict_proba['gbt'] = gbt_prob
    predict_proba['gnn'] = gnn_prob
    predict_lr_pred = consensus_model.predict(predict_proba)
    return predict_lr_pred

In [6]:
gnn_model = torch.load('models/attentive_fp.pt')
rf_rdk_model = load_model('rf','RDK')
svm_ecfp_model = load_model('svm','ECFP')
lgb_rdk_model = load_model('lgb','RDK')
xgb_maccs_model = load_model('xgb','MACCS')
gbt_maccs_model = load_model('gbt','MACCS')

/home/cflou/anaconda3/envs/dgl/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/home/cflou/anaconda3/envs/dgl/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/home/cflou/anaconda3/envs/dgl/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator SVC from version 1.0 when using version 1.0.2. This might le

In [7]:
#dgm_raw_filename = "/home/cflou/Project/ames/model/data/ames_predict/ames_pos_new_comps.csv"
dgm_raw_filename = "data/external_set.csv"
dgm_feature_filename = dgm_raw_filename.replace('.csv','.pickle')
dgm_filename = dgm_raw_filename.replace('.csv','')
dgm_smiles_tasks_df = pd.read_csv(dgm_raw_filename)
dgm_smilesList = dgm_smiles_tasks_df.smiles.values
dgm_remained_smiles = []
dgm_canonical_smiles_list = []
for smiles in dgm_smilesList:
    try:        
        mol = Chem.MolFromSmiles(smiles)
        dgm_canonical_smiles_list.append(Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True))
        dgm_remained_smiles.append(smiles)
    except:
        print("not successfully processed smiles: ", smiles)
        pass
print("number of successfully processed smiles: ", len(dgm_remained_smiles))
dgm_smiles_tasks_df = dgm_smiles_tasks_df[dgm_smiles_tasks_df["smiles"].isin(dgm_remained_smiles)]
dgm_smiles_tasks_df['cano_smiles'] =dgm_canonical_smiles_list
dgm_smilesList = [smiles for smiles in dgm_canonical_smiles_list]
if os.path.isfile(dgm_feature_filename):
    dgm_feature_dicts = pickle.load(open(dgm_feature_filename, "rb" ))
else:
    dgm_feature_dicts = save_smiles_dicts(dgm_smilesList,dgm_filename)
    
dgm_remained_df = dgm_smiles_tasks_df[dgm_smiles_tasks_df["cano_smiles"].isin(dgm_feature_dicts['smiles_to_atom_mask'].keys())]
dgm_uncovered_df = dgm_smiles_tasks_df.drop(dgm_remained_df.index)
print(str(len(dgm_uncovered_df.cano_smiles))+' compounds cannot be featured')
dgm_remained_df = dgm_remained_df.reset_index(drop=True)

number of successfully processed smiles:  1469
0 compounds cannot be featured


In [8]:
dgm_con_pred = consensus_model_predict(gnn_model,rf_rdk_model,svm_ecfp_model,lgb_rdk_model,xgb_maccs_model,gbt_maccs_model,
                                     dgm_remained_df
                                    )

feature dicts file saved as predict_data.pickle


/home/cflou/anaconda3/envs/dgl/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [9]:
from rdkit.Chem import rdMolDescriptors
from rdkit import DataStructs

app_do = pd.read_csv('models/applicability_domain.csv')
molset_train = [Chem.MolFromSmiles(smi) for smi in app_do.smiles]
fps_train = [rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024) for mol in molset_train]
ad = 0.0880+4*0.0625

def application_domain(ad,dataset,fps_train):
    ap_domain = []
    for smi in dataset.smiles:
        mol = Chem.MolFromSmiles(smi)
        fps_test = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
        sim = []
        for j in fps_train:
            similarity=DataStructs.FingerprintSimilarity(fps_test,j)
            sim.append(similarity)
        top = sorted(sim,reverse = True)[0:5]
        if min(top) > ad:
            ap_domain.append(1)
        else:
            ap_domain.append(0)
    return ap_domain

ap = application_domain(ad,dgm_remained_df,fps_train)
dgm_remained_df['pred_values'] = dgm_con_pred
dgm_remained_df['ad'] = ap

In [10]:
dgm_remained_df.to_csv('predict_results/external_results.csv')